In [4]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.authentication import InteractiveLoginAuthentication
import json
import os

In [5]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [6]:
os.makedirs('src', exist_ok=True)
os.makedirs('artifacts', exist_ok=True)

## Create Config File

In [7]:
prefix = 'Dummy'

cfg = {
    'experiment_name' : prefix+'Prediction',
    'env_name' : 'WILO_POC_'+prefix,
    'symptomcodes_dataset' : 'symptomcodes.csv',
    'raw_data_dataset' : 'ItemResourceData.csv',
    'prepared_data_dataset' : prefix+'PreparedData',
    'train_dataset' : prefix+'TrainData',
    'test_dataset' : prefix+'TestData',
    'PreprocessPipeline' : prefix+'PreprocessPipeline',
    'TrainedClassifier' : prefix+'Classifier',
    'TrainedRegressor' : prefix+'Regressor',
    'webservice_name' : prefix.lower()+'-webservice',
    'compute_target' : 'mlcompute',
    'inference_target' : 'mlinference',
    'storage': 'workspaceblobstore'
}

In [8]:
# save to file
with open("experiment_cfg.json", "w") as cfg_file:
    json.dump(cfg, cfg_file)

## Define Environment

In [ ]:
# define dependencies

cd = CondaDependencies()
cd.add_conda_package('numpy==1.16.2')
cd.add_pip_package("azureml-defaults")
cd.add_pip_package('azureml-dataprep[pandas,fuse]')
cd.add_pip_package('pyarrow==0.12.0')
cd.add_pip_package('joblib==0.14.1')
cd.add_pip_package('scikit-learn==0.20.3')

In [ ]:
# register environment

if cfg['env_name'] in ws.environments:
    env = ws.environments[cfg['env_name']]
else:
    env = Environment(cfg['env_name'])
env.python.conda_dependencies = cd
env.docker.enabled = True
env.register(ws)